In [1]:
import os

import numpy as np

from sklearn.preprocessing import OneHotEncoder

# Load data from each folder in /data/toPreProc/

data_in = 0 # Number of input dimensions to the model
data_out = 0 # Number of output dimensions to the model

data_dict = {}
label_dict = {}

for folder in os.listdir('data/toPreProc/'):
    print("Folder name: ", folder)
    data_dict[folder] = np.load(f'data/toPreProc/{folder}/{folder}PreprocessedData.npy')
    labels = np.load(f'data/toPreProc/{folder}/{folder}Labels.npy', allow_pickle=True)
    label_dict[folder] = labels[:-1, 2]
    print("Data shape: ", data_dict[folder].shape)
    print("Label shape: ", label_dict[folder].shape)
    if(data_dict[folder].shape[1] > data_in):
        data_in = data_dict[folder].shape[1]

# # One hot encode the labels
# ohes = {}
# num_of_labels = len(label_dict) # Includes one extra for the end of the sequence
# for key, y in label_dict.items():
#     #

#     #
#     print(y.reshape(-1, 1))
#     ohes[key] = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
#     label_dict[key] = ohes[key].transform(y)
#     print(label_dict[key])

Folder name:  2023-12-18_003
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-15_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-17_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-17_001
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-18_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-18_001
Data shape:  (26, 100, 68)
Label shape:  (26,)


In [8]:
import torch
import random

import torch.nn as nn

# Define the sequential model
model = nn.Sequential(
    nn.Conv1d(kernel_size=5, padding_mode="replicate", in_channels=data_in, out_channels=64),

    nn.Conv1d(kernel_size=3, padding_mode="replicate", in_channels=64, out_channels=64)

)